In [1]:
from datasets import Dataset
import sys

sys.path.append('../src')
from read_data import get_essay_sets, essays_by_set
from essay_scoring_model import EssayScoringModel

c:\Users\elikl\Documents\Uni\yr3\LLMs\Automatic-SAS-with-LLMs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load train and test data for a specific essay set
essay_set = 1
print(f"Processing Essay Set {essay_set}")

train_data = list(essays_by_set(essay_set))
train_dataset = Dataset.from_dict({
    "EssayText": [x['EssayText'] for x in train_data],
    "Id": [x['Id'] for x in train_data],
    "EssaySet": [x['EssaySet'] for x in train_data],
    "Score1": [x['Score1'] for x in train_data],
})
test_data = list(essays_by_set(essay_set, "../data/public_leaderboard_rel_2.tsv"))
test_data_scores = list(essays_by_set(essay_set, "../data/public_leaderboard_solution.csv"))

test_scores_dict = {x['Id']: x['Score1'] for x in test_data_scores}
test_scores = [test_scores_dict[x['Id']] for x in test_data]

test_dataset = Dataset.from_dict({
    "EssayText": [x['EssayText'] for x in test_data],
    "Id": [x['Id'] for x in test_data],
    "EssaySet": [x['EssaySet'] for x in test_data],
    "Score1": test_scores,
})

# Tokenize (and save) training and test data
unique_labels = len(set(x['Score1'] for x in train_data))
scoring_model = EssayScoringModel(model_name='bert-base-uncased', num_labels=unique_labels)
tokenized_train_dataset = scoring_model.get_tokenized_dataset(train_dataset, is_train=True, essay_set=essay_set, batch_size=8)
tokenized_test_dataset = scoring_model.get_tokenized_dataset(test_dataset, is_train=False, essay_set=essay_set, batch_size=8)

Processing Essay Set 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading tokenized dataset from ../data/tokenized_set1_train
Loading tokenized dataset from ../data/tokenized_set1_test


In [3]:
evaluation_results = scoring_model.train(tokenized_train_dataset, tokenized_test_dataset, output_dir=f'./results_{essay_set}', batch_size=8, epochs=3)
print(f"Evaluation results for essay set {essay_set}: {evaluation_results}")

c:\Users\elikl\Documents\Uni\yr3\LLMs\Automatic-SAS-with-LLMs\.venv\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/627 [00:00<?, ?it/s]

ValueError: Target size (torch.Size([8])) must be the same as input size (torch.Size([8, 4]))

In [5]:
def main():
    model_name = 'bert-base-uncased'
    batch_size = 8
    epochs = 3

    scoring_model = EssayScoringModel(model_name=model_name)

    for essay_set in get_essay_sets():
        print(f"Processing Essay Set {essay_set}")

        # Load train and test data for the specific essay set
        train_data = list(essays_by_set(essay_set))
        train_dataset = HFDataset.from_dict({
            "EssayText": [x['EssayText'] for x in train_data],
            "Score1": [x['Score1'] for x in train_data],
            "Id": [x['Id'] for x in train_data],
            "EssaySet": [x['EssaySet'] for x in train_data],
        })
        test_data = list(essays_by_set(essay_set, "../data/public_leaderboard_rel_2.tsv"))
        test_dataset = HFDataset.from_dict({
            "EssayText": [x['EssayText'] for x in test_data],
            "Id": [x['Id'] for x in test_data],
            "EssaySet": [x['EssaySet'] for x in test_data],
        })

        # Tokenize (and save) training and test data
        tokenized_train_dataset = scoring_model.tokenize_and_save(train_dataset, is_train=True, essay_set=essay_set, batch_size=batch_size)
        tokenized_test_dataset = scoring_model.tokenize_and_save(test_dataset, is_train=False, essay_set=essay_set, batch_size=batch_size)

        # Train the model and evaluate
        evaluation_results = scoring_model.train(tokenized_train_dataset, tokenized_test_dataset, output_dir=f'./results_{essay_set}', batch_size=batch_size, epochs=epochs)
        print(f"Evaluation results for essay set {essay_set}: {evaluation_results}")